# Import and dependencies


In [ ]:
# Install dependencies
!pip install opencv-python tqdm glob2 numpy Pillow scipy


In [1]:
from lxml import etree as letree
import cv2
import tqdm
import glob
import os
import random
import numpy as np

from PIL import Image, ImageDraw
from scipy.ndimage import rotate


In [ ]:
'''
for folder in sorted(glob.glob('export_job_*/*/')):
    for path in sorted(glob.glob(f'{folder}CH_ACV_P_RENE_MONOD_*/*.jpg')):

        # ICI METTRE TOUT LE RESTE DU CODE (AVEC INDENTATION) => y compris les partitions (cellule suivante)

    cahier_nom = path.split('/')[-1]

    os.environ('mv -v BenthamDatasetR0-GT handwritten_text_recognition/raw/bentham/)
    os.environ('python handwritten_text_recognition/src/main.py --source=bentham --transform')
    os.environ(f'mv -v handwritten_text_recognition/data/bentham.hdf5 {cahier_nom}.hdf5')
    os.environ('rm -rf handwritten_text_recognition/raw/bentham/BenthamDatasetR0-GT')
'''


# Functions


In [2]:
def baseline2box(pts, up: int = 20, down: int = 5, margin_l: int = 5, margin_r: int = 5):
    min_y = np.min(pts[:, 0])-margin_l
    max_y = np.max(pts[:, 0])+margin_r
    min_x = np.min(pts[:, 1])-up
    max_x = np.max(pts[:, 1])+down

    return min_y, max_y, min_x, max_x


def checkCoords(min_x, max_x, min_y, max_y):
    return int(np.max([min_x, 0])), max_x, int(np.max([min_y, 0])), max_y


def filterImage(gray_image):
    fimage = cv2.bilateralFilter(gray_image, 7, 50, 50)
    value = np.ravel(fimage)
    paper, ink = np.percentile(value, 95), np.percentile(value, 5)

    white_fimage = (fimage-ink)/(paper-ink)*255
    white_fimage[white_fimage > 255] = 255
    white_fimage[white_fimage < 1] = 0
    white_fimage = white_fimage.astype('uint8')

    return white_fimage


def formatList(list_):
    string = ''
    for item in list_:
        string += f'{item}\n'
    return string


In [ ]:
#probleme 93-98 + 101 + 106 + 108 + 111 + 113 + 116 + 117 + 118 + 121 + 122 + 123 + 126 + 127 + 128 + 130 + 131 + 132 + 133 
# + 139 + 144 + 148 + 149 + 150 + 157 + 158 + 159 + 160 + 161 + 162 + 163 + 166 + 180 + 171 + 174 + 176 + 177 + 180 + 186 
# + 187 + 210 

#fait : 93 94 - 98 - 108 111 113 116 117 118 121 122 123 126 

In [3]:
pad = 60


list_notebooks = []

for path in tqdm.tqdm(sorted(glob.glob('Cahiers_Polier_Complet_XML/CH_ACV_P_RENE_MONOD_18*/*.jpg'))):
        
        # Rest the list_segments for each notebook
    list_segments = []

        # Load image and recover its name
    image = filterImage(cv2.imread(path, 0))
    image_name = path.split("/")[-1][:-4]
    notebook, page_n = image_name.split('_')[-1].split('-')

        # Create directory for each notebook
    os.makedirs(f'Bentham_{notebook}/BenthamDatasetR0-GT/Images/Lines/', exist_ok=True)
    os.makedirs(f'Bentham_{notebook}/BenthamDatasetR0-GT/Partitions/', exist_ok=True)
    os.makedirs(f'Bentham_{notebook}/BenthamDatasetR0-GT/Transcriptions/', exist_ok=True)

        # A TESTER EN ENLEVANT
        #list_notebooks.append(notebook)



        # Read Transkribus XML output
    tree = letree.parse(
        f'{path.split("/CH_ACV")[0]}/CH_ACV_P_RENE_MONOD_{notebook}/page/CH_ACV_P_RENE_MONOD_{notebook}-{page_n}.xml')
    root = tree.getroot()
    page = root.find(
            '{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}Page')

        # Iterate over text lines detected on the page
    counter = 0
    for text_region in page.iter('{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}TextRegion'):
        for text_line in text_region.iter('{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}TextLine'):



                # Transkribus 'box' coords
                coords = text_line.find(
                    '{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}Coords')
                coords = [double.split(',')
                          for double in coords.attrib['points'].split(' ')]
                coords = np.array([(int(coord[0]), int(coord[1]))
                                  for coord in coords]).astype('uint32')

                # Transkribus baseline coords
                baseline = text_line.find(
                    '{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}Baseline')
                if baseline is None:
                    continue

                baseline = [double.split(
                    ',') for double in baseline.attrib['points'].split(' ')]
                baseline = np.array([(int(coord[0]), int(coord[1]))
                                    for coord in baseline]).astype('uint32')

                # Extend the 'box'  coords
                min_y, max_y, min_x, max_x = baseline2box(
                    baseline, up=76, down=25, margin_l=18, margin_r=31)
                min_yp, max_yp, min_xp, max_xp = checkCoords(
                    min_y-pad, max_y+pad, min_x-pad, max_x+pad)

                # Select the line region
                segment = image[min_xp:max_xp, min_yp:max_yp]
                mask = np.zeros(segment.shape[:2], np.uint8)

                # Compute the line rotation angle
                cv2.drawContours(
                    mask, [coords - np.array([min_yp, min_xp])], -1, (255, 255, 255), -1, cv2.LINE_AA)
                _, _, theta = cv2.minAreaRect(coords.astype('int'))
                if theta > 30:
                    theta = theta-90

                # Compensate the rotation
                rot_segment = rotate(segment, theta, cval=0)
                rot_mask = rotate(mask, theta, cval=0).astype('bool')

                # Compute the area of interest
                try:
                    min_xm, max_xm = (np.arange(rot_mask.shape[0])[(np.sum(rot_mask, 1)/rot_mask.shape[1]) > 0.1])[
                        np.array([0, -1])]
                    min_ym, max_ym = (np.arange(rot_mask.shape[1])[(np.sum(rot_mask, 0)/rot_mask.shape[0]) > 0.05])[
                        np.array([0, -1])]
                except IndexError:
                    print (IndexError)
                    continue

                # quand inférence si ça fonctionne pas utiliser cette ligne ;
                #text_segment = text_line.find(
                #        '{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}TextEquiv')[0].text
                #text_segment = text_segment.replace('é', 'e').replace('è', 'e').replace('ö', 'o').replace('î', 'i').replace('ï', 'i').replace('ê', 'e').replace('ù','u').replace('à','a').replace('ç','c').replace('ò','o').replace('û','u').replace('ä','a')
                text_segment = 'sans transcription' 

                # Save transcription
                with open(f'Bentham_{notebook}/BenthamDatasetR0-GT/Transcriptions/{notebook}_{page_n}_{counter}.txt', 'w+') as f:
                    f.write(text_segment)

                # Save segment image
                cv2.imwrite(f'Bentham_{notebook}/BenthamDatasetR0-GT/Images/Lines/{notebook}_{page_n}_{counter}.png',
                            rot_segment[max([0, min_xm-34]):max_xm+15, max([0, min_ym-18]):max_ym+10])
                list_segments.append(f'{notebook}_{page_n}_{counter}')
                counter += 1

    val, test = len(list_segments)*0.1, len(list_segments)*0.2
    random.shuffle(list_segments)


    with open(f'Bentham_{notebook}/BenthamDatasetR0-GT/Partitions/TestLines.lst', 'a') as f:
        f.write(formatList(list_segments))
        f.close()
    with open(f'Bentham_{notebook}/BenthamDatasetR0-GT/Partitions/TrainLines.lst', 'a') as f:
        f.write(formatList(list_segments[1:4]))
        f.close()
    with open(f'Bentham_{notebook}/BenthamDatasetR0-GT/Partitions/ValidationLines.lst', 'a') as f:
        f.write(formatList(list_segments[4:7]))
        f.close()


 31%|███       | 123/401 [13:34<25:49,  5.57s/it]

<class 'IndexError'>


 32%|███▏      | 129/401 [14:11<25:24,  5.60s/it]

<class 'IndexError'>


 33%|███▎      | 134/401 [14:39<22:09,  4.98s/it]

...

 38%|███▊      | 151/401 [16:14<25:52,  6.21s/it]

<class 'IndexError'>


 43%|████▎     | 174/401 [19:09<32:18,  8.54s/it]

<class 'IndexError'>


100%|██████████| 401/401 [34:43<00:00,  5.20s/it]
